<a href="https://colab.research.google.com/github/krakowiakpawel9/data-science-bootcamp/blob/master/07_uczenie_glebokie/02_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* @author: krakowiakpawel9@gmail.com  
* @site: e-smartdata.org

### Tensorflow
>Strona biblioteki: [https://www.tensorflow.org/](https://www.tensorflow.org/)  
>Dokumentacja: [https://www.tensorflow.org/api_docs](https://www.tensorflow.org/api_docs)  
>
>Szybko rozwijająca się biblioteka do uczenia maszynowego i uczenia głębokiego rozwijana przez Google.
>
>Aby zainstalować bibliotekę tensorflow, użyj polecenia poniżej:
```
pip install tensorflow
```

### Keras
>Strona biblioteki: [https://keras.io/](https://keras.io/)  
>
>Wysokopoziomowy interfejs do budowy sieci neuronowych. Działa zarówno na Tensorflow, Theano oraz CNTK.
>
>Aby zainstalować bibliotekę keras, użyj polecenia poniżej:
```
pip install keras
```

### Model sekwencjny (Sequential Model):
1. [Wprowadzenie - pierwszy model](#a0)
2. [Funkcje aktywacji](#a1)
3. [Kompilacja modelu](#a2)
4. [Trenowanie modelu](#a3)
5. [Przykład - klasyfikacja binarna](#a4)
6. [Przykład - klasyfikacja wieloklasowa](#a5)
7. [Przykład - regresja](#a6)



Załadowanie biblioteki tensorflow


In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import pandas as pd
import plotly.express as px
tf.__version__

TensorFlow 2.x selected.


'2.0.0'

### <a name='a0'></a>Wprowadzenie - pierwszy model
Model sekwencyjny to nic innego jak liniowy stos warstw.

In [0]:
# utworzenie instancji klasy Sequential
from tensorflow.keras.models import Sequential

model = Sequential()
print(model)

Podstawowym elementem składowym modelu są warstwy. Aby dodać najbardziej standardową warstwę - warstwę gęsto połączoną należy użyć warstwy **Dense**. Aby dodać warstwę do modelu należy użyć metody *.add()*

In [0]:
from tensorflow.keras.layers import Dense

model.add(Dense(units=4, input_shape=(10,)))

Wyświetlenie podsumowania modelu: metoda *.summary()*

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 44        
Total params: 44
Trainable params: 44
Non-trainable params: 0
_________________________________________________________________


Dodanie kolejnej warstwy

In [0]:
model.add(Dense(units=2))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 44        
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 10        
Total params: 54
Trainable params: 54
Non-trainable params: 0
_________________________________________________________________


### <a name='a1'></a>Funkcje aktywacji
Istotnym elemenem sieci neuronowych jest dobór odpowiednich funkcji aktywacji. Funkcje aktywacji jak sama nazwa wskazuje są odpowiedzialne za aktywowanie odpowiednich neuronów podczas procesu uczenia.

Jeżeli nie określimy podczas dodawania warstwy funkcji aktywacji, domyślnie stosowana jest liniowa funkcja aktywacji, tzn. $a(x)=x$

Warstwa z liniową funkcją aktywacji może uczyć się tylko liniowych przekształceń danych wejściowych. Dlatego stosuje się różne funkcje aktywacji aby rozwiazywać problemy nieliniowe.

[Keras: Funkcje aktywacji](https://keras.io/activations/)

In [0]:
from tensorflow.keras.activations import linear

random_data = np.linspace(start=-3, stop=3, num=300)
data = pd.DataFrame({'data': random_data, 'linear': linear(random_data)})
data.head()

,data,linear
0,-3.000000,-3.000000
1,-2.979933,-2.979933
2,-2.959866,-2.959866
3,-2.939799,-2.939799
4,-2.919732,-2.919732


In [0]:
px.line(data, x='data', y='linear', width=500, height=400, range_y=[-3, 3])

In [0]:
from tensorflow.keras.activations import sigmoid

data = pd.DataFrame({'data': random_data, 'sigmoid': sigmoid(random_data)})
data.head()

,data,sigmoid
0,-3.000000,0.047426
1,-2.979933,0.048341
2,-2.959866,0.049272
3,-2.939799,0.050221
4,-2.919732,0.051187


In [0]:
px.line(data, x='data', y='sigmoid', width=500, height=400, range_y=[-0.5, 1.5])

In [0]:
from tensorflow.keras.activations import relu

data = pd.DataFrame({'data': random_data, 'relu': relu(random_data)})
data.head()

,data,relu
0,-3.000000,0.0
1,-2.979933,0.0
2,-2.959866,0.0
3,-2.939799,0.0
4,-2.919732,0.0


In [0]:
px.line(data, x='data', y='relu', width=500, height=400, range_y=[-0.5, 1.5])

In [0]:
from tensorflow.keras.activations import tanh

data = pd.DataFrame({'data': random_data, 'tanh': tanh(random_data)})
data.head()

,data,tanh
0,-3.000000,-0.995055
1,-2.979933,-0.994853
2,-2.959866,-0.994643
3,-2.939799,-0.994424
4,-2.919732,-0.994196


In [0]:
px.line(data, x='data', y='tanh', width=500, height=400, range_y=[-1.5, 1.5])

In [0]:
model = Sequential()
model.add(Dense(units=8, activation='relu', input_shape=(10,)))
model.add(Dense(units=1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 97
Trainable params: 97
Non-trainable params: 0
_________________________________________________________________


### <a name='a2'></a>Kompilacja modelu
Przed rozpoczęciem trenowania sieci należy odpowiednio skonfigurować proces uczenia. W tym kroku określamy:
* rodzaj optymalizatora ([Keras - Optymalizatory](https://keras.io/optimizers/)) 
* funkcję straty ([Keras - Funkcje Straty](https://keras.io/losses/))
* metryki, które będziemy obserwować podczas trenowania sieci ([Keras - Metryki](https://keras.io/metrics/))

In [0]:
# klasyfikacja binarna
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# klasyfikacja wieloklasowa
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# regresja
model.compile(optimizer='rmsprop',
              loss='mse')

### <a name='a3'></a>Trenowanie modelu
Za dane wejściowe do modelu należy przekazać Numpy arrays:
* **epochs** - krotność przejścia danych przez sieć w procesie uczenia
* **batch_size** - rozmiar wsadu po którym następuje aktualizacja wag
* **validation_split** - część danych treningowych, które zostaną wykorzystane jako zbiór walidacyjny
* **validation_data** - (x_val, y_val) - dane wykorzystane do walidacji modelu

In [0]:
# model.fit(data, labels, epochs=10, batch_size=32)
# model.fit(data, labels, epochs=10, batch_size=32, validation_split=0.2)
# model.fit(data, labels, epochs=10, batch_size=32, validation_data=(x_val, y_val))

### <a name='a4'></a> Przykład - klasyfikacja binarna

In [0]:
data = np.random.randn(10000, 150)
labels = np.random.randint(2, size=(10000, 1))

print(data.shape)
print(labels.shape)

(10000, 150)
(10000, 1)


In [0]:
data[:3]

In [0]:
labels[:10]

array([[0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [0]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20)

Train on 10000 samples
Epoch 1/20
10000/10000 [==============================] - 2s 177us/sample - loss: 0.7472 - accuracy: 0.4958
Epoch 2/20
10000/10000 [==============================] - 1s 53us/sample - loss: 0.6961 - accuracy: 0.5396
Epoch 3/20
10000/10000 [==============================] - 1s 54us/sample - loss: 0.6786 - accuracy: 0.5693
Epoch 4/20
10000/10000 [==============================] - 1s 52us/sample - loss: 0.6652 - accuracy: 0.5936
Epoch 5/20
10000/10000 [==============================] - 1s 53us/sample - loss: 0.6526 - accuracy: 0.6183
Epoch 6/20
10000/10000 [==============================] - 1s 54us/sample - loss: 0.6393 - accuracy: 0.6360
Epoch 7/20
10000/10000 [==============================] - 1s 53us/sample - loss: 0.6249 - accuracy: 0.6566
Epoch 8/20
10000/10000 [==============================] - 1s 53us/sample - loss: 0.6113 - accuracy: 0.6743
Epoch 9/20
10000/10000 [==============================] - 1s 53us/sample - loss: 0.5959 - accuracy: 0.6879
Epoch 10/20
1

In [0]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20, batch_size=64)

Train on 10000 samples
Epoch 1/20
10000/10000 [==============================] - 1s 70us/sample - loss: 0.7522 - accuracy: 0.5069
Epoch 2/20
10000/10000 [==============================] - 0s 28us/sample - loss: 0.7010 - accuracy: 0.5396
Epoch 3/20
10000/10000 [==============================] - 0s 28us/sample - loss: 0.6834 - accuracy: 0.5678
Epoch 4/20
10000/10000 [==============================] - 0s 28us/sample - loss: 0.6711 - accuracy: 0.5866
Epoch 5/20
10000/10000 [==============================] - 0s 29us/sample - loss: 0.6596 - accuracy: 0.6063
Epoch 6/20
10000/10000 [==============================] - 0s 27us/sample - loss: 0.6489 - accuracy: 0.6277
Epoch 7/20
10000/10000 [==============================] - 0s 29us/sample - loss: 0.6382 - accuracy: 0.6488
Epoch 8/20
10000/10000 [==============================] - 0s 28us/sample - loss: 0.6272 - accuracy: 0.6633
Epoch 9/20
10000/10000 [==============================] - 0s 27us/sample - loss: 0.6161 - accuracy: 0.6773
Epoch 10/20
10

In [0]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/20
8000/8000 [==============================] - 1s 134us/sample - loss: 0.7480 - accuracy: 0.5017 - val_loss: 0.7293 - val_accuracy: 0.4955
Epoch 2/20
8000/8000 [==============================] - 1s 64us/sample - loss: 0.6948 - accuracy: 0.5371 - val_loss: 0.7210 - val_accuracy: 0.5075
Epoch 3/20
8000/8000 [==============================] - 0s 60us/sample - loss: 0.6746 - accuracy: 0.5782 - val_loss: 0.7222 - val_accuracy: 0.5010
Epoch 4/20
8000/8000 [==============================] - 0s 62us/sample - loss: 0.6584 - accuracy: 0.6061 - val_loss: 0.7229 - val_accuracy: 0.5110
Epoch 5/20
8000/8000 [==============================] - 0s 59us/sample - loss: 0.6439 - accuracy: 0.6338 - val_loss: 0.7271 - val_accuracy: 0.5080
Epoch 6/20
8000/8000 [==============================] - 0s 61us/sample - loss: 0.6284 - accuracy: 0.6509 - val_loss: 0.7354 - val_accuracy: 0.5020
Epoch 7/20
8000/8000 [==============================] - 0s 62us/sampl

In [0]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2, verbose=0)

In [0]:
metrics = history.history
metrics.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [0]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

Train on 8000 samples, validate on 2000 samples
Epoch 1/20
8000/8000 [==============================] - 1s 115us/sample - loss: 0.7533 - accuracy: 0.4941 - val_loss: 0.7171 - val_accuracy: 0.5040
Epoch 2/20
8000/8000 [==============================] - 0s 61us/sample - loss: 0.6981 - accuracy: 0.5371 - val_loss: 0.7145 - val_accuracy: 0.4970
Epoch 3/20
8000/8000 [==============================] - 1s 65us/sample - loss: 0.6795 - accuracy: 0.5658 - val_loss: 0.7145 - val_accuracy: 0.5035
Epoch 4/20
8000/8000 [==============================] - 0s 60us/sample - loss: 0.6642 - accuracy: 0.5996 - val_loss: 0.7204 - val_accuracy: 0.4975
Epoch 5/20
8000/8000 [==============================] - 0s 61us/sample - loss: 0.6499 - accuracy: 0.6223 - val_loss: 0.7246 - val_accuracy: 0.4940
Epoch 6/20
8000/8000 [==============================] - 0s 60us/sample - loss: 0.6351 - accuracy: 0.6480 - val_loss: 0.7293 - val_accuracy: 0.4920
Epoch 7/20
8000/8000 [==============================] - 1s 65us/sampl

In [0]:
test_data = np.random.randn(5, 150)
test_labels = np.random.randint(2, size=(5, 1))

In [0]:
model.predict(test_data)

array([[0.7386832 ],
       [0.29738083],
       [0.14625075],
       [0.61528796],
       [0.35780835]], dtype=float32)

In [0]:
model.predict_proba(test_data)

array([[0.7386832 ],
       [0.29738083],
       [0.14625075],
       [0.61528796],
       [0.35780835]], dtype=float32)

In [0]:
model.predict_classes(test_data)

array([[1],
       [0],
       [0],
       [1],
       [0]], dtype=int32)

### <a name='a5'></a> Przykład - klasyfikacja wieloklasowa

In [0]:
data = np.random.random((10000, 150))
labels = np.random.randint(10, size=(10000, 1))

In [0]:
print(data.shape)
print(labels.shape)

(10000, 150)
(10000, 1)


In [0]:
labels[:10]

array([[2],
       [6],
       [8],
       [3],
       [0],
       [3],
       [9],
       [6],
       [7],
       [0]])

In [0]:
from tensorflow.keras.utils import to_categorical
labels = to_categorical(labels, num_classes=10)
labels

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [0]:
labels[1]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)

In [0]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(units=10, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, batch_size=32, epochs=30, validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/30
8000/8000 [==============================] - 1s 140us/sample - loss: 2.3235 - accuracy: 0.0980 - val_loss: 2.3153 - val_accuracy: 0.0985
Epoch 2/30
8000/8000 [==============================] - 0s 60us/sample - loss: 2.3052 - accuracy: 0.1047 - val_loss: 2.3040 - val_accuracy: 0.0995
Epoch 3/30
8000/8000 [==============================] - 1s 63us/sample - loss: 2.3031 - accuracy: 0.1009 - val_loss: 2.3104 - val_accuracy: 0.1090
Epoch 4/30
8000/8000 [==============================] - 1s 63us/sample - loss: 2.3016 - accuracy: 0.1039 - val_loss: 2.3061 - val_accuracy: 0.1065
Epoch 5/30
8000/8000 [==============================] - 0s 60us/sample - loss: 2.3005 - accuracy: 0.1155 - val_loss: 2.3084 - val_accuracy: 0.0990
Epoch 6/30
8000/8000 [==============================] - 0s 61us/sample - loss: 2.2979 - accuracy: 0.1119 - val_loss: 2.3134 - val_accuracy: 0.0985
Epoch 7/30
8000/8000 [==============================] - 0s 62us/sampl

In [0]:
test_data = np.random.random((10, 150))

model.predict(test_data)

array([[0.09911948, 0.11620636, 0.07320527, 0.15997937, 0.08313195,
        0.11985189, 0.07282171, 0.08499081, 0.09132902, 0.09936419],
       [0.10808039, 0.11060798, 0.1585828 , 0.04076305, 0.11867085,
        0.11622953, 0.10032436, 0.10682581, 0.06562724, 0.07428803],
       [0.0535747 , 0.0826021 , 0.15126106, 0.04798588, 0.03644897,
        0.08196826, 0.17952028, 0.07557219, 0.14473361, 0.14633295],
       [0.12001421, 0.15575112, 0.13602467, 0.08515704, 0.06826283,
        0.06773269, 0.0736742 , 0.08663882, 0.09184736, 0.11489701],
       [0.1503428 , 0.14404033, 0.20681618, 0.0435313 , 0.07448085,
        0.05074822, 0.06210018, 0.0954838 , 0.08630197, 0.08615443],
       [0.09830751, 0.07327876, 0.17003337, 0.14070372, 0.07893839,
        0.0776796 , 0.12669246, 0.09419221, 0.07801841, 0.06215557],
       [0.05894386, 0.06890954, 0.10082155, 0.08828751, 0.06637203,
        0.16312268, 0.15596974, 0.08681998, 0.09833211, 0.11242106],
       [0.07287443, 0.06215156, 0.2424068

In [0]:
model.predict_classes(test_data)

array([3, 2, 6, 1, 2, 2, 5, 2, 4, 2])

### <a name='a6'></a> Przykład - regresja

In [0]:
data = np.random.random((10000, 150))
labels = 50 * np.random.random(10000)

In [0]:
labels[:10]

array([ 2.76589765, 26.25951681, 42.46470588, 33.45842006,  6.10047945,
       42.16472461,  0.06071142,  9.63306066,  7.97318495, 49.39214105])

In [0]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(units=1))

model.compile(optimizer='rmsprop',
              loss='mse')

model.fit(data, labels, epochs=30, batch_size=32, validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/30
8000/8000 [==============================] - 1s 99us/sample - loss: 269.9293 - val_loss: 207.3698
Epoch 2/30
8000/8000 [==============================] - 0s 60us/sample - loss: 208.6766 - val_loss: 207.3182
Epoch 3/30
8000/8000 [==============================] - 0s 57us/sample - loss: 208.1491 - val_loss: 208.2676
Epoch 4/30
8000/8000 [==============================] - 0s 58us/sample - loss: 207.8588 - val_loss: 207.4424
Epoch 5/30
8000/8000 [==============================] - 0s 58us/sample - loss: 207.5430 - val_loss: 207.8541
Epoch 6/30
8000/8000 [==============================] - 0s 58us/sample - loss: 207.2514 - val_loss: 209.6800
Epoch 7/30
8000/8000 [==============================] - 0s 60us/sample - loss: 206.8425 - val_loss: 208.7884
Epoch 8/30
8000/8000 [==============================] - 0s 59us/sample - loss: 206.7675 - val_loss: 207.5888
Epoch 9/30
8000/8000 [==============================] - 0s 58us/sample - loss: 2

In [0]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(units=1))

model.compile(optimizer='rmsprop',
              loss='mae',
              metrics=['mse'])

model.fit(data, labels, epochs=10, batch_size=32, validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 1s 107us/sample - loss: 13.3688 - mse: 250.6315 - val_loss: 12.4057 - val_mse: 207.8904
Epoch 2/10
8000/8000 [==============================] - 0s 58us/sample - loss: 12.4745 - mse: 209.1429 - val_loss: 12.4120 - val_mse: 208.6593
Epoch 3/10
8000/8000 [==============================] - 1s 64us/sample - loss: 12.4606 - mse: 208.5854 - val_loss: 12.3944 - val_mse: 207.4827
Epoch 4/10
8000/8000 [==============================] - 0s 58us/sample - loss: 12.4351 - mse: 208.3050 - val_loss: 12.4262 - val_mse: 209.8297
Epoch 5/10
8000/8000 [==============================] - 0s 58us/sample - loss: 12.4279 - mse: 207.8135 - val_loss: 12.4113 - val_mse: 208.1427
Epoch 6/10
8000/8000 [==============================] - 0s 59us/sample - loss: 12.4111 - mse: 207.7288 - val_loss: 12.4030 - val_mse: 207.9031
Epoch 7/10
8000/8000 [==============================] - 0s 62us/sample - loss: 12.3957 - mse:

In [0]:
test_data = np.random.random((10, 150))

model.predict(test_data)

array([[25.318836],
       [25.124805],
       [26.249212],
       [26.726828],
       [24.240023],
       [26.99018 ],
       [22.903202],
       [25.071842],
       [22.852095],
       [25.543777]], dtype=float32)